# 核心模块组成
1. 文本编码器（Text Encoder）
采用模型：Stable Diffusion v2 使用OpenCLIP（取代 v1 的 CLIP Text）作为文本编码器。
功能：将输入的文本提示（Prompt）转换为数值化的语义向量，捕捉文本深层语义，用于指导图像生成。
输出特征：77 个 token 嵌入向量，每个向量维度为 768 维。
核心作用：通过对比学习框架精准关联文本与图像语义，提升对复杂文本描述的理解能力（如 “宇宙海滩” 的视觉元素匹配）。
2. 自动编码器（Autoencoder）
由编码器（Encoder） 和解码器（Decoder） 组成，实现图像与潜在空间的双向转换，是 “感知压缩（Perceptual Compression）” 的核心。
3. 扩散模型（UNet + Scheduler）
在潜在空间中执行逐步去噪过程，是生成图像的核心模块。
（1）UNet 结构
输入：带噪声的潜在表示（4 通道）、文本嵌入向量（来自文本编码器）、时间步信息。
核心功能：通过多层卷积与注意力机制预测并去除噪声，输出优化后的潜在表示。
结构细节：
下采样模块（Down Blocks）：逐步降低空间分辨率，增加通道数，提取高层语义特征；包含 ResnetBlock2D（残差连接，缓解梯度消失）和下采样层（stride=2 的卷积）。
中间模块（Mid Block）：在最低分辨率下融合时间步和文本条件信息，通过自注意力和交叉注意力增强全局上下文理解。
上采样模块（Up Blocks）：逐步恢复空间分辨率，减少通道数；通过跳跃连接（Skip Connection）融合下采样阶段的细节特征。
交叉注意力机制：将文本嵌入与图像特征交互，确保生成内容与文本描述对齐。
（2）调度器（Scheduler）
功能：控制去噪步数和噪声强度，平衡生成速度与质量。
常用类型：DDIM（Denoising Diffusion Implicit Models）为默认调度器，支持快速采样（10-50 步），确定性更新路径提升生成稳定性。
4. 附加功能模块（v2 新增）
Depth-to-Image：根据深度图控制生成图像的透视与层次感，扩展创作自由度。
超分辨率增强：从低分辨率输入生成高清图像，提升细节表现力。

# 整体工作流程
文本编码：输入文本提示经 OpenCLIP 编码为 768 维语义向量。
潜在扩散初始化：随机生成与潜在空间维度匹配的噪声向量（如 4x64x64）。
逐步去噪：UNet 在 Scheduler 控制下，结合文本嵌入和时间步信息，对噪声向量逐步去噪（50-100 步），生成优化后的潜在表示。
图像解码：自动编码器的解码器将潜在表示还原为 RGB 图像。